In [1]:
import numpy  as np  # type: ignore
import pandas as pd # type: ignore
from scipy.optimize import linprog # type: ignore

In [2]:
suppliers      = ["Ashley", 
                  "Bedford", 
                  "Consol", 
                  "Dunby", 
                  "Earlam", 
                  "Florence", 
                  "Gaston", 
                  "Hopt"]
price          = [49.5, 50.0, 
                  61.0, 63.5, 
                  66.5, 71.0, 
                  72.5, 80.0]  # Cost per ton
is_union       = [1, 1, 0, 1, 0, 1, 0, 0]                          # Union mine: 1 = Yes, 0 = No
transport_mode = ["Rail", "Truck", 
                  "Rail", "Truck", 
                  "Truck", "Truck", 
                  "Rail", "Rail"]
volatility     = [15, 16, 18, 20, 21, 22, 23, 25]                  # Volatility percent
capacity       = [300, 600, 510, 655, 575, 680, 450, 490]          # Max tons each supplier can deliver


In [ ]:
c    = price
A_eq = [[1] * 8]
b_eq = [1225]
A_ub = []
b_ub = []
A_ub.append([-v for v in volatility])
b_ub.append(-232.75)
A_ub.append([-u for u in is_union])
b_ub.append(-612.5)
A_ub.append([1 if m == "Rail" else 0 for m in transport_mode])
b_ub.append(650)

In [4]:
A_ub.append([1 if m == "Truck" else 0 for m in transport_mode])
b_ub.append(720)
bounds = [(0, cap) for cap in capacity]

In [5]:
print("Running optimization...")
result = linprog(c=c, A_ub=A_ub, 
                      b_ub=b_ub, 
                      A_eq=A_eq, 
                      b_eq=b_eq,
                      bounds=bounds, 
                      method='highs')

Running optimization...


In [ ]:
if result.success:
    purchased    = result.x
    total_cost   = np.dot(purchased, price)
    average_cost = total_cost / 1225
    df = pd.DataFrame({
        "Supplier": suppliers,
        "Price": price,
        "Purchased (mtons)": purchased,
        "Cost": [p * x for p, x in zip(price, purchased)]
    })
    print("\nOptimal Purchase Plan:")
    print(df.round(2).to_string(index=False))
    print(f"\nTotal Cost          : ${total_cost:,.2f}")
    print(f"Average Cost per Ton: ${average_cost:.2f}")
else:
    print("Optimization failed:", result.message)


Optimal Purchase Plan:
Supplier  Price  Purchased (mtons)    Cost
  Ashley   49.5              300.0 14850.0
 Bedford   50.0              600.0 30000.0
  Consol   61.0              325.0 19825.0
   Dunby   63.5                0.0     0.0
  Earlam   66.5                0.0     0.0
Florence   71.0                0.0     0.0
  Gaston   72.5                0.0     0.0
    Hopt   80.0                0.0     0.0

Total Cost          : $64,675.00
Average Cost per Ton: $52.80
